# Chapter 6: Decision Trees

---

## 📖 Rangkuman Chapter 6

Chapter ini membahas **Decision Trees**, salah satu algoritme Machine Learning yang paling serbaguna. Seperti SVM, Decision Trees dapat melakukan tugas klasifikasi, regresi, dan bahkan multioutput.

Mereka adalah algoritme yang kuat, mampu menangani dataset yang kompleks, dan merupakan komponen dasar dari **Random Forests**, salah satu model ML paling kuat yang ada saat ini. Chapter ini akan mencakup cara melatih, memvisualisasikan, dan membuat prediksi dengan Decision Trees, serta algoritme CART, regularisasi, dan keterbatasannya.

## 🎯 Topics Covered

| No | Topic | Description |
|----|-------|-------------|
| 1 | **Training & Visualizing** | Melatih `DecisionTreeClassifier` dan memvisualisasikannya |
| 2 | **Making Predictions** | Cara kerja pohon dalam mengambil keputusan (root-to-leaf) |
| 3 | **White Box vs. Black Box** | Mengapa Decision Trees mudah diinterpretasi |
| 4 | **Estimating Probabilities** | Cara pohon menghasilkan probabilitas kelas |
| 5 | **CART Algorithm** | Algoritme *greedy* yang digunakan Scikit-Learn untuk melatih pohon |
| 6 | **Gini vs. Entropy** | Dua metode untuk mengukur "impurity" (kemurnian) node |
| 7 | **Regularization** | Cara mencegah overfitting (misalnya, `max_depth`, `min_samples_leaf`) |
| 8 | **Regression** | Menggunakan Decision Trees untuk tugas regresi (`DecisionTreeRegressor`) |
| 9 | **Limitations** | Keterbatasan utama: sensitivitas terhadap rotasi dan variasi data |

## 🌳 Training & Visualizing a Decision Tree

Kita dapat melatih `DecisionTreeClassifier` pada dataset Iris hanya dengan beberapa baris kode.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

# Melatih model dengan membatasi kedalaman maksimum (regularisasi)
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

print("Model Decision Tree telah dilatih!")

Setelah dilatih, pohon ini dapat divisualisasikan menggunakan `export_graphviz()` dari Scikit-Learn.

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

print("File 'iris_tree.dot' telah dibuat.")

*(File `.dot` ini kemudian dapat dikonversi menjadi PNG menggunakan alat Graphviz).*

## 🧭 Making Predictions

Proses prediksi sangat intuitif, mirip dengan permainan "20 Questions".

1.  Anda mulai dari **root node** (node paling atas, depth 0). Node ini bertanya: "Apakah petal length <= 2.45 cm?"
2.  Jika **True**, Anda pindah ke node anak kiri (depth 1, left). Node ini adalah **leaf node** (tidak punya anak). Pohon memprediksi `class=setosa`.
3.  Jika **False** (petal length > 2.45 cm), Anda pindah ke node anak kanan (depth 1, right).
4.  Node ini **bukan leaf node**, jadi ia bertanya lagi: "Apakah petal width <= 1.75 cm?"
5.  Jika **True**, pohon memprediksi `class=versicolor` (depth 2, left).
6.  Jika **False**, pohon memprediksi `class=virginica` (depth 2, right).

💡 **Kualitas Utama:** Salah satu keunggulan Decision Trees adalah mereka **tidak memerlukan feature scaling atau centering** sama sekali.

### Atribut Node

* `samples`: Menghitung berapa banyak instance pelatihan yang "melewati" node tersebut.
* `value`: Menunjukkan berapa banyak instance pelatihan dari setiap kelas yang ada di node ini.
* `gini`: Mengukur **impurity** (ketidakmurnian) node.
    * Node "murni" ($gini=0$) jika semua instance di dalamnya milik satu kelas yang sama.

### Gini Impurity

Skor Gini dihitung menggunakan formula berikut:
$$G_{i}=1-\sum_{k=1}^{n}{p_{i,k}}^{2}$$
* $p_{i,k}$ adalah rasio (proporsi) instance kelas $k$ di antara total instance di node $i$.

## 📦 White Box vs. Black Box

Decision Trees dianggap sebagai model **"white box"**.
* ✅ **White Box:** Model ini intuitif, dan keputusan yang diambilnya mudah untuk diinterpretasi. Anda dapat dengan jelas melihat aturan-aturan yang digunakan untuk membuat prediksi (misalnya, "JIKA petal_length <= 2.45 MAKA setosa").

Sebaliknya, Random Forests atau Neural Networks dianggap sebagai model **"black box"**.
* ❌ **Black Box:** Meskipun prediksinya hebat, biasanya sangat sulit untuk menjelaskan *mengapa* model tersebut membuat prediksi tertentu.

## 📊 Estimating Class Probabilities

Decision Tree juga dapat memperkirakan probabilitas sebuah instance milik kelas $k$.

1.  Pertama, pohon menelusuri jalurnya untuk menemukan leaf node untuk instance tersebut.
2.  Kemudian, ia mengembalikan **rasio instance pelatihan** dari kelas $k$ di leaf node itu.

## 🧮 CART Algorithm

Scikit-Learn menggunakan algoritme **Classification and Regression Tree (CART)** untuk melatih Decision Trees.

Ide dasar CART adalah algoritme *greedy*: Ia mencari split optimal pada level atas, kemudian mengulang proses ini pada level berikutnya. (Tidak memeriksa apakah split akan menghasilkan biaya terendah secara keseluruhan—ini adalah pendekatan "greedy" yang tidak menjamin solusi optimal, tetapi cukup baik dan sederhana.)

**CART Cost Function (Classification):**
$$J(k,t_{k})=\frac{m_{left}}{m}G_{left}+\frac{m_{right}}{m}G_{right}$$
* $G_{left/right}$ = Gini impurity dari subset kiri/kanan.
* $m_{left/right}$ = Jumlah instance di subset kiri/kanan.

**Complexity:**
* [cite_start]Membuat prediksi: $O(\log_2(m))$ (balanced tree). [cite: 854]
* [cite_start]Pelatihan: $O(n \times m \log_2(m))$ (n = fitur, m = instance). [cite: 801]

## Gini vs. Entropy

Scikit-Learn menggunakan **Gini impurity** secara default (`criterion="gini"`), tapi Anda bisa mengubahnya menjadi **entropy** (`criterion="entropy"`).

**Entropy Impurity:**
$$H_{i}=-\sum_{k=1}^{n}p_{i,k}\log_{2}(p_{i,k})$$
* Node dengan entropy=0 adalah "murni" (semua instance sama kelas).
* Hampir sama dengan Gini, tapi menghasilkan pohon yang sedikit lebih seimbang.

**Gini vs. Entropy:**
* Gini lebih cepat dihitung (tidak ada log).
* Perbedaan praktisnya kecil; keduanya menghasilkan pohon yang mirip.

## Regularization

Decision Trees membuat sangat sedikit asumsi tentang data (non-parametric), sehingga jika tidak dibatasi, model cenderung **overfit**.

**Hyperparameters Regularization:**
* `max_depth`: Batas kedalaman pohon.
* `min_samples_split`: Minimum sampel untuk split node.
* `min_samples_leaf`: Minimum sampel di leaf node.
* `max_leaf_nodes`: Batas jumlah leaf nodes.
* `max_features`: Jumlah fitur yang dievaluasi untuk setiap split.

Meningkatkan `min_*` atau mengurangi `max_*` akan meregularisasi model (mengurangi overfitting).

## Regression

Decision Trees juga bisa digunakan untuk regresi (`DecisionTreeRegressor`).

Prediksi: Alih-alih memprediksi kelas, regresi memprediksi **nilai rata-rata** dari instance di leaf node.

**CART for Regression:**
Algoritme sama, tetapi alih-alih meminimalkan *impurity* (Gini/Entropy), ia mencoba membagi data untuk meminimalkan **MSE (Mean Squared Error)**.

**CART Cost Function (Regression):**
$$J(k,t_{k})=\frac{m_{left}}{m}MSE_{left}+\frac{m_{right}}{m}MSE_{right}$$
* Di mana $MSE_{node} = \frac{1}{m_{node}} \sum_{i \in \text{node}} (\hat{y}_{node} - y^{(i)})^2$

Sama seperti klasifikasi, Decision Tree Regressor juga sangat rentan terhadap **overfitting** jika tidak diregularisasi (misalnya, dengan `min_samples_leaf=10`).

## 📉 Instability & Limitations

Meskipun kuat, Decision Trees memiliki beberapa kelemahan:

1.  **Sensitivitas terhadap Rotasi Data**
    * Decision Trees menyukai batas keputusan yang **ortogonal** (tegak lurus dengan sumbu fitur).
    * Ini membuat mereka sangat sensitif terhadap rotasi data. Jika data diputar (misalnya 45 derajat), pohon akan membuat batas keputusan "tangga" yang rumit dan tidak akan menggeneralisasi dengan baik.

2.  **Sensitivitas terhadap Variasi Data (Instabilitas)**
    * Isu utama Decision Trees adalah mereka **sangat sensitif terhadap variasi kecil** dalam data pelatihan.
    * Misalnya, jika Anda hanya menghapus satu instance (bunga Iris versicolor terlebar) dari dataset, Anda mungkin mendapatkan pohon yang terlihat sangat berbeda.
    * Algoritme Scikit-Learn juga bersifat *stochastic* (acak), sehingga Anda bisa mendapatkan model yang berbeda bahkan pada data yang sama (kecuali Anda mengatur `random_state`).

💡 **Solusi:** **Random Forests** (dibahas di Bab 7) dapat mengatasi instabilitas ini dengan mengambil rata-rata prediksi dari banyak pohon.

## 🔧 Exercises (from the book)

### Exercise 1
**Q:** What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances?
**A:** $O(\log_2(m))$. Untuk $m = 1.000.000$, $\log_2(10^6) \approx 19.93$. Jadi, kedalamannya sekitar **20**.

---

### Exercise 2
**Q:** Is a node's Gini impurity generally lower or greater than its parent's? Is it generally lower/greater, or *always* lower/greater?
**A:** **Generally lower**. Algoritme CART (Eq. 6-2) secara eksplisit mencari split yang meminimalkan *weighted average* Gini impurity dari anak-anaknya. Karena itu, *weighted average* Gini *selalu* lebih rendah. Namun, Gini satu *anak* (child) bisa saja lebih tinggi dari induknya, asalkan *total weighted average*-nya lebih rendah.

---

### Exercise 3
**Q:** If a Decision Tree is overfitting the training set, is it a good idea to try decreasing `max_depth`?
**A:** **Ya**. Mengurangi `max_depth` adalah salah satu cara utama untuk meregularisasi model dan mengurangi overfitting.

---

### Exercise 4
**Q:** If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?
**A:** **Tidak**. Decision Trees tidak sensitif terhadap penskalaan fitur, jadi itu tidak akan membantu. Jika underfitting, Anda harus *meningkatkan* kompleksitas (misalnya, *meningkatkan* `max_depth` atau *mengurangi* `min_samples_leaf`).

---

### Exercise 5
**Q:** If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?
**A:** Kompleksitas pelatihan adalah $O(n \times m \log_2(m))$. 
Rasio waktu $\approx \frac{10M \log_2(10M)}{1M \log_2(1M)} \approx 10 \times \frac{23.3}{19.9} \approx 11.7$.
Jadi, akan memakan waktu sekitar **11.7 jam**.

---

### Exercise 6
**Q:** If your training set contains 100,000 instances, will setting `presort=True` speed up training?
**A:** **Tidak**. Presorting hanya mempercepat pelatihan pada dataset kecil (kurang dari beberapa ribu instance) dan akan **memperlambat** pelatihan secara signifikan pada dataset yang lebih besar.

---

### Exercise 7
**Q:** Train and fine-tune a Decision Tree for the moons dataset...
**A:** (Requires implementation)
* `make_moons(n_samples=10000, noise=0.4)`
* `train_test_split()`
* `GridSearchCV` untuk `DecisionTreeClassifier`, coba berbagai nilai `max_leaf_nodes`.
* Latih pada set penuh dan ukur performa (target ~85-87% akurasi).

---

### Exercise 8
**Q:** Grow a forest...
**A:** (Requires implementation)
* Buat 1.000 subset (masing-masing 100 instance) dari training set (gunakan `ShuffleSplit`).
* Latih 1.000 Decision Trees menggunakan hyperparameter terbaik dari Ex. 7.
* Evaluasi 1.000 pohon di test set (performa mungkin turun).
* Untuk setiap instance test set, ambil prediksi mayoritas (majority vote) dari 1.000 pohon (gunakan `scipy.stats.mode()`).
* Evaluasi prediksi *majority-vote* ini. Akurasi seharusnya meningkat sedikit (0.5-1.5%) dibanding model pertama.

---

**Happy Splitting! 🌳**